In [2]:
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import os

from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import backend as K

In [ ]:
# Credits: https://github.com/kairess/fingerprint_recognition/blob/master/train.ipynb

def palm_model():
    x1 = layers.Input(shape=(90, 90, 1))
    x2 = layers.Input(shape=(90, 90, 1))

    # share weights both inputs
    inputs = layers.Input(shape=(90, 90, 1))
    x = layers.Conv2D(32, kernel_size=3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(32, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    feature_model = Model(inputs=inputs, outputs=x, name='feature_model')
    
    x1_net = feature_model(x1)
    x2_net = feature_model(x2)
    
    # subtract features
    net = layers.Subtract(name='subtract')([x1_net, x2_net])

    net = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu', name='diff_conv_net')(net)
    net = layers.MaxPooling2D(pool_size=2, name='maxpool_net')(net)
    net = layers.Flatten(name='flatten_net')(net)
    net = layers.Dropout(0.5)(net)
    net = layers.Dense(64, activation='relu', name='relu_net')(net)
    net = layers.Dropout(0.5)(net)
    net = layers.Dense(1, activation='sigmoid', name='sigmoid_net')(net)
    model = Model(inputs=[x1, x2], outputs=net)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def get_dataset(start_idx, end_idx, batch_size=128, is_training=False):
    train_df = pd.read_csv("datasets/train.csv", usecols=['palm_print', 'label'])
    val_df = pd.read_csv("datasets/val.csv", usecols=['palm_print', 'label'])
    df = pd.concat([train_df, val_df])
    df = df[df.label.isin(list(range(start_idx, end_idx)))]
    df.drop_duplicates(inplace=True)
    df = df.join(df, lsuffix="_left", rsuffix="_right", how='cross')
    df = df[df.palm_print_left != df.palm_print_right]
    pos_df = df[df.label_left == df.label_right]
    neg_df = df[df.label_left != df.label_right]
    neg_df = neg_df.sample(n=min(pos_df.shape[0], neg_df.shape[0]))
    neg_df = neg_df.sample(n=min(pos_df.shape[0], neg_df.shape[0]))
    balanced_df = pd.concat([pos_df, neg_df]).sample(frac=1)
    print("Total no. of records:", balanced_df.shape[0])
    print("No. of similar pairs:", balanced_df[balanced_df.label_left == balanced_df.label_right].shape[0])
    print("No. of distin. pairs:", balanced_df[balanced_df.label_left != balanced_df.label_right].shape[0])
    print("No. of batches:", balanced_df.shape[0]//batch_size)
    def process(dataframe):
        def read_img():
            for idx, row in dataframe.iterrows():
                left_img = cv2.resize(cv2.imread(row.palm_print_left, 0), (90, 90))/255.
                right_img = cv2.resize(cv2.imread(row.palm_print_right, 0), (90, 90))/255.
                label = 0 if row.label_left == row.label_right else 1
                yield (np.expand_dims(left_img, axis=-1), np.expand_dims(right_img, axis=-1)), label
        return read_img

    dataset = tf.data.Dataset.from_generator(process(balanced_df), output_types=((np.float32, np.float32), np.int8),
                                            output_shapes=(((90, 90, 1), (90, 90, 1)), []))
    if is_training:
        dataset = dataset.repeat().shuffle(buffer_size=7000)
    dataset = dataset.batch(batch_size)
    return dataset

In [ ]:
for d in get_dataset(0, 150, batch_size=128).take(1):
    print(d[1])

In [ ]:
batch_size = 128
train_dataset = get_dataset(0, 150, batch_size=batch_size, is_training=True)
val_dataset = get_dataset(150, 200, batch_size=batch_size)
test_dataset = get_dataset(200, 210, batch_size=batch_size)

In [ ]:
model = palm_model()

In [ ]:
epochs = 3
steps_per_epoch = 50
validation_steps = 3

history = model.fit(train_dataset, epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_dataset, validation_steps=validation_steps)

In [ ]:
model.predict(test_dataset)

In [ ]:
features_model = Model(model.get_layer('feature_model').input, outputs=model.get_layer('feature_model').output)
# features_model.summary()

In [ ]:
diff_model = Sequential()
diff_model.add(layers.Input(shape=model.get_layer("diff_conv_net").input.shape[1:]))
diff_model.add(model.get_layer("diff_conv_net"))
diff_model.add(model.get_layer("maxpool_net"))
diff_model.add(model.get_layer("flatten_net"))
diff_model.add(model.get_layer("relu_net"))
diff_model.add(model.get_layer("sigmoid_net"))
diff_model = Model(inputs=diff_model.inputs, outputs=diff_model.outputs)

In [ ]:
features_model.save("models/palm_features_model.h5")
diff_model.save("models/palm_diff_model.h5")
model.save("models/palm_siamese_model.h5")

In [ ]:
model = load_model("models/palm_siamese_model.h5")

In [ ]:
# eval_dataset = get_dataset(0, 200, batch_size=128)
# model.evaluate(eval_dataset)

In [ ]:
test_dataset = get_dataset(250, 252, batch_size=128)
model.evaluate(test_dataset)
model.predict(test_dataset)

In [ ]:
file_1 = "datasets/PalmCropped/100/0100_m_l_03.jpg"
file_2 = "datasets/PalmCropped/101/0101_m_l_04.jpg"

def preproc(filename):
    img = np.expand_dims(cv2.resize(cv2.imread(file_1, 0), (90, 90)), (0, -1))
    return img

img_1 = preproc(file_1)
img_2 = preproc(file_2)

feat_1 = features_model.predict(img_1)
feat_2 = features_model.predict(img_2)
sub_feat = feat_2 - feat_1
prediction = diff_model.predict(sub_feat)
print(prediction)

In [ ]:
sub_feat = feat_1 - feat_2
prediction = diff_model.predict(sub_feat)
print(prediction)

In [3]:
from models_utils import face_model
from preproc_utils import face_preprocess

In [16]:
import numpy as np
import cv2

from keras_vggface.utils import preprocess_input
from scipy.spatial.distance import cosine
from PIL import Image

# Face
def face_preprocess(file, size=(224, 224)):
    image = cv2.imread(file)[::-1]
    image = cv2.resize(image, size)
    face = np.expand_dims(image, axis=0).astype(np.float32)
    face = preprocess_input(face, version=2)
    return face

In [17]:
def get_encodings(model, file):
    face = face_preprocess(file, (224, 224))
    yhat = model.predict(face)
    return yhat

In [18]:
file_1 = "datasets/PalmCropped/100/0100_m_l_03.jpg"
file_2 = "datasets/PalmCropped/100/0100_m_l_04.jpg"
file_3 = "datasets/PalmCropped/101/0101_m_l_04.jpg"

In [19]:
model = face_model()
enc_1 = get_encodings(model, file_1)
enc_2 = get_encodings(model, file_2)
enc_3 = get_encodings(model, file_3)

In [20]:
print(cosine(enc_1, enc_2))
print(cosine(enc_1, enc_3))
print(cosine(enc_2, enc_3))

0.007586956024169922
0.07879728078842163
0.06955486536026001


In [21]:
from glob import glob
files = sorted(list(glob("datasets/PalmCropped/*/*.jpg")), key=lambda x: os.path.basename(x))
files = files[:100]

In [ ]:
for file_1, file_2 in zip(files, files[1:]):
    enc_1 = get_encodings(model, file_1)
    enc_2 = get_encodings(model, file_2)
    print(cosine(enc_1, enc_2)>0.01, 0 if os.path.basename(os.path.dirname(file_1)) ==
                                      os.path.basename(os.path.dirname(file_2)) else 1)

True 0
True 0
False 0
True 0
True 1
False 0
False 0
True 0
False 0
True 1
False 0
False 0
True 0
True 0
True 1
